In [16]:
from typing import *

import os
import json
from pathlib import Path

import httpx
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
    api_key=os.environ.get("Kimi_API_KEY"),  # 在这里将 MOONSHOT_API_KEY 替换为你从 Kimi 开放平台申请的 API Key
    base_url="https://api.moonshot.cn/v1",
)


def upload_files(files: List[str], cache_tag: Optional[str] = None) -> List[Dict[str, Any]]:
    messages = []
    for file in files:
        file_object = client.files.create(file=Path(file), purpose="file-extract")
        file_content = client.files.content(file_id=file_object.id).text
        messages.append({
            "role": "system",
            "content": file_content,
        })

    if cache_tag:
        r = httpx.post(f"{client.base_url}/caching",
                       headers={
                           "Authorization": f"Bearer {client.api_key}",
                       },
                       json={
                           "model": "moonshot-v1",
                           "messages": messages,
                           "ttl": 300,
                           "tags": [cache_tag],
                       })

        if r.status_code != 200:
            raise Exception(r.text)

        return [{
            "role": "cache",
            "content": f"tag={cache_tag};reset_ttl=300",
        }]
    else:
        return messages


def main():
    file_messages = upload_files(
        files=["test files/象上-文稿-转写结果.docx"],
        cache_tag="upload_files",
    )

    quant_dict = {
        'quant_idea': [],
        'factor_composition': [],
        'factor_explanation': [],
        'model_explanation': [],
        'factor_update': [],
        'strategy_execution': []
    }

    Strategy_prompt_dic = {
        "Simple Quant Strategy": [
            '管理⼈量化策略的整体思路, 例如，是偏强逻辑为主, 对因⼦的可解释性有很⾼的要求；还是偏暴力挖掘为主，使用机器学习较多',
            '从文件中提取出管理人的策略中的因子组成和因子数量,包括量价类,基本面，另类因子占比多少，以及因子的总数',
            '从文件中提取出管理人对其不同因子类型的解释（如有），具体来说主要是不同因子是基于哪些数据得出的，不要编造，只能抽取提供信息中有的内容。如果没有详细说明，则返回无',
            '从文件中提取出管理人的建模方法，需要明确提示是线性还是机器学习模型或者是两者都有及其偏好。其中线性模型包括：排序打分和简单回归，详述他们的建模方法论, '
            '并提取出他们使用了的算法和用途。不要把因子挖掘的部分写进去，只需要说建模',
            '从文件中提取出管理人的新因子入库条件，包括但不限于在模拟盘中观察的时间, 衡量指标如夏普，ICIR等，以及其他的措施等。不要编造，只能抽取提供信息中有的内容。如果没有详细说明，则返回无',
            '从文件中提取出管理人的具体操作，例如有无使用自研/外部的下单算法，不同策略的持股数是多少，年化双边换手率多少倍，行业偏离度，风格偏离度，barra风格因子暴露等'
        ]
    }

    # 逐条提取并分类信息
    for prompt, component in zip(Strategy_prompt_dic["Simple Quant Strategy"], list(quant_dict.keys())):
        messages = [
            *file_messages,
            {
                "role": "system",
                "content": prompt,
            },
        ]
        
        completion = client.chat.completions.create(
            model="moonshot-v1-128k",
            messages=messages,
        )

        reply = completion.choices[0].message.content
        quant_dict[component].append(reply)

    print(json.dumps(quant_dict, indent=2, ensure_ascii=False))
    print(quant_dict)

if __name__ == '__main__':
    main()


Exception: {"code":5,"error":"url.not_found","message":"没找到对象","method":"POST","scode":"0x5","status":false,"ua":"python-httpx/0.27.0","url":"/v1//caching"}